In [ ]:
#page 1-7 Train data for Recognition Model

#50_big -- training data for LLM
#test_big -- test data for LLM
#val_big -- val data for LLM






In [7]:
import csv
from aksharamukha import transliterate
import os
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
import random
import ast
import re

In [8]:
repeat_data_number = 1
experiment = f'messy_transferCheck_{repeat_data_number}x'
synthetic_data = True

#CHECK IF POST CORRECTION MODEL TRAINED ON BIG TRANSFER:
#/home/ocr_proj/OCR/v09042024/line_images/test_clean
#/home/ocr_proj/OCR/v09042024/line_images/test_messy
test_path = "/home/ocr_proj/OCR/v09042024/line_images/test_messy"


In [9]:

directory = f'/home/ocr_proj/OCR/post_correction/pe-ocr-sanskrit/data/experiment_{experiment}'
if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"Directory created: {directory}")
else:
    print(f"Directory already exists: {directory}")

Directory created: /home/ocr_proj/OCR/post_correction/pe-ocr-sanskrit/data/experiment_messy_transferCheck_1x


In [10]:
def natural_sort_key(s):
    return [int(c) if c.isdigit() else c.lower() for c in re.split(r'(\d+)', s)]

In [11]:
def process_directory(directory_path, output_dir, output_name):
    # Read the contents of labels.txt and pred.txt
    with open(os.path.join(directory_path, 'labels.txt'), 'r', encoding='utf-8') as f:
        target_texts = f.read().replace('||','॥').replace('।।','॥').replace('|','।').splitlines()
    
    with open(os.path.join(directory_path, 'pred.txt'), 'r', encoding='utf-8') as f:
        input_texts = f.read().replace('||','॥').replace('।।','॥').replace('|','।').splitlines()
    
    # Get the list of jpg files
    jpg_files = sorted([f for f in os.listdir(directory_path) if f.endswith('.jpg')], key=natural_sort_key)
    
    # Create the output data
    output_data = []
    for i, jpg_file in enumerate(jpg_files):
        devanagari_input_line = input_texts[i]
        devanagari_target_line = ''.join(target_texts[i].split()[1:])
        output_data.append({
            'input_text': transliterate.process('Devanagari', 'IAST', devanagari_input_line),
            'target_text': transliterate.process('Devanagari', 'IAST', devanagari_target_line),
            'path': os.path.join(directory_path, jpg_file)
        })
    
    # Write the output to a CSV file
    output_file = os.path.join(output_dir, output_name)
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['input_text', 'target_text', 'path']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames,delimiter=';')
        
        writer.writeheader()
        for row in output_data:
            writer.writerow(row)
    
    print(f"CSV file created: {output_file}")

In [12]:
#process_directory(train_path,directory,output_name='train_fold_1.csv')
process_directory(test_path,directory,output_name='test_fold_1.csv')
#process_directory(val_path,directory,output_name='val_fold_1.csv')

CSV file created: /home/ocr_proj/OCR/post_correction/pe-ocr-sanskrit/data/experiment_messy_transferCheck_1x/test_fold_1.csv
